In [1]:
from abstractions.goap.actions import Action, Prerequisites, Consequences
from abstractions.goap.entity import Attribute, Statement
from abstractions.goap.spatial import GameEntity, Node, Position, Path, ActionInstance
from typing import List, Dict, Any, Optional, Tuple, Callable


In [2]:

class Door(GameEntity):
    open: Attribute = Attribute(name="open", value=False)
    blocks_movement: Attribute = Attribute(name="blocks_movement", value=True)
    blocks_light: Attribute = Attribute(name="blocks_light", value=True)

    def __init__(self, **data):
        super().__init__(**data)
        self.update_block_attributes()

    def update_block_attributes(self):
        self.blocks_movement = Attribute(name="blocks_movement", value=not self.open.value)
        self.blocks_light = Attribute(name="blocks_light", value=not self.open.value)

class Character(GameEntity):
    can_act: Attribute = Attribute(name="can_act", value=True)

# Define the open_door action
open_door = Action(
    name="Open Door",
    prerequisites=Prerequisites(
        source_statements=[Statement(conditions={"can_act": True})],
        target_statements=[Statement(conditions={"open": False})]
    ),
    consequences=Consequences(
        target_transformations={"open": True}
    )
)

# Define the close_door action
close_door = Action(
    name="Close Door",
    prerequisites=Prerequisites(
        source_statements=[Statement(conditions={"can_act": True})],
        target_statements=[Statement(conditions={"open": True})]
    ),
    consequences=Consequences(
        target_transformations={"open": False}
    )
)

# Create a test environment
node1 = Node(name="Node 1", position=Position(value=(0, 0)))
node2 = Node(name="Node 2", position=Position(value=(1, 0)))

door = Door(name="Wooden Door")
character = Character(name="John")

node1.add_entity(door)
node1.add_entity(character)

print("Initial state:")
print(f"Door open: {door.open.value}")
print(f"Door blocks movement: {door.blocks_movement.value}")
print(f"Door blocks light: {door.blocks_light.value}")
print(f"Node 1 blocks movement: {node1.blocks_movement}")
print(f"Node 1 blocks light: {node1.blocks_light}")
print()

# Test the open_door action
if open_door.is_applicable(character, door):
    updated_character, updated_door = open_door.apply(character, door)
    # updated_door.update_block_attributes()
    # node1.update_entity(door, updated_door)
    print("After opening the door:")
    print(f"Door open: {updated_door.open.value}")
    print(f"Door blocks movement: {updated_door.blocks_movement.value}")
    print(f"Door blocks light: {updated_door.blocks_light.value}")
    print(f"Node 1 blocks movement: {node1.blocks_movement}")
    print(f"Node 1 blocks light: {node1.blocks_light}")
else:
    print("Cannot open the door")
print()

# Test the close_door action
if close_door.is_applicable(character, updated_door):
    updated_character, updated_updated_door = close_door.apply(character, updated_door)
    # node1.update_entity(updated_door, updated_updated_door)
    print("After closing the door:")
    print(f"Door open: {updated_updated_door.open.value}")
    print(f"Door blocks movement: {updated_updated_door.blocks_movement.value}")
    print(f"Door blocks light: {updated_updated_door.blocks_light.value}")
    print(f"Node 1 blocks movement: {node1.blocks_movement}")
    print(f"Node 1 blocks light: {node1.blocks_light}")
else:
    print("Cannot close the door")

Initial state:
Door open: False
Door blocks movement: True
Door blocks light: True
Node 1 blocks movement: True
Node 1 blocks light: True

After opening the door:
Door open: True
Door blocks movement: False
Door blocks light: False
Node 1 blocks movement: False
Node 1 blocks light: False

After closing the door:
Door open: False
Door blocks movement: True
Door blocks light: True
Node 1 blocks movement: True
Node 1 blocks light: True


In [3]:
from abstractions.goap.actions import Action, Prerequisites, Consequences
from abstractions.goap.entity import Attribute, Statement
from abstractions.goap.spatial import GameEntity, Node, Position, GridMap, ActionsPayload, ActionInstance, ActionsResults


# Create a test environment
node1 = Node(name="Node 1", position=Position(value=(0, 0)))
node2 = Node(name="Node 2", position=Position(value=(1, 0)))
door = Door(name="Wooden Door")
character = Character(name="John")
node1.add_entity(door)
node1.add_entity(character)

grid_map = GridMap(width=2, height=1)
grid_map.set_node((0, 0), node1)
grid_map.set_node((1, 0), node2)

print("Initial state:")
print(f"Door open: {door.open.value}")
print(f"Door blocks movement: {door.blocks_movement.value}")
print(f"Door blocks light: {door.blocks_light.value}")
print(f"Node 1 blocks movement: {node1.blocks_movement}")
print(f"Node 1 blocks light: {node1.blocks_light}")
print()

# Create an ActionsPayload using entity IDs
actions_payload = ActionsPayload(actions=[
    ActionInstance(source_id=character.id, target_id=door.id, action=open_door),
    ActionInstance(source_id=character.id, target_id=door.id, action=close_door)
])

# Apply the actions payload to the grid map
actions_results = grid_map.apply_actions_payload(actions_payload)

# Print the results
for result in actions_results.results:
    print(f"Action: {result.action_instance.action.name}")
    print(f"Success: {result.success}")
    if not result.success:
        print(f"Error: {result.error}")
    print()

# Print the final state
print("Final state:")
print(f"Door open: {door.open.value}")
print(f"Door blocks movement: {door.blocks_movement.value}")
print(f"Door blocks light: {door.blocks_light.value}")
print(f"Node 1 blocks movement: {node1.blocks_movement}")
print(f"Node 1 blocks light: {node1.blocks_light}")

Initial state:
Door open: False
Door blocks movement: True
Door blocks light: True
Node 1 blocks movement: True
Node 1 blocks light: True

Action: Open Door
Success: True

Action: Close Door
Success: True

Final state:
Door open: False
Door blocks movement: True
Door blocks light: True
Node 1 blocks movement: True
Node 1 blocks light: True


In [4]:
from abstractions.goap.actions import Action, Prerequisites, Consequences
from abstractions.goap.entity import Attribute, Statement
from abstractions.goap.spatial import GameEntity, Node, Position, GridMap, ActionsPayload, ActionInstance, ActionsResults, Path, BlocksMovement, BlocksLight

def source_node_comparison(source: Node, target: Node) -> bool:

    return source in target.neighbors()

def target_walkable_comparison(source: GameEntity, target: GameEntity) -> bool:
    return not target.blocks_movement.value

def move_to_target_node(source: GameEntity, target: GameEntity) -> Node:
    return target.node

MoveToTargetNode: Callable[[GameEntity, GameEntity], Node] = move_to_target_node

class MoveStep(Action):
    name: str = "Move Step"
    prerequisites: Prerequisites = Prerequisites(
        source_statements=[Statement(conditions={"can_act": True})],
        target_statements=[Statement(conditions={"blocks_movement": False})],
        source_target_statements=[Statement(comparisons={
            "source_position": ("node", "node", source_node_comparison)
        })]
    )
    consequences: Consequences = Consequences(
        source_transformations={"node": MoveToTargetNode}
    )


# Create a simple grid map with two nodes


grid_map = GridMap(width=2, height=1)


# Create a player and a floor entity
class Character(GameEntity):
    can_act: Attribute = Attribute(name="can_act", value=True)

player = Character(name="Player",blocks_movement=BlocksMovement(value=False))
floor = GameEntity(name="Floor", blocks_movement=BlocksMovement(value=False))

# Add the player to node1 and the floor to node2
node1 = grid_map.get_node((0, 0))
node2 = grid_map.get_node((1, 0))
node1.add_entity(player)
node2.add_entity(floor)
print("player_id",player.id)
print("floor_id",floor.id)

# Create an ActionInstance for the MoveStep action
action_instance = ActionInstance(source_id=player.id, target_id=floor.id, action=MoveStep())
action_instance
actions_payload = ActionsPayload(actions=[action_instance])

# Apply the actions payload to the grid map
actions_results = grid_map.apply_actions_payload(actions_payload)

# Print the results
for result in actions_results.results:
    print(f"Action: {result.action_instance.action.name}")
    print(f"Success: {result.success}")
    if not result.success:
        print(f"Error: {result.error}")
updated_player = GameEntity.get_instance(player.id)


player_id be18c600-b55e-4a81-a7aa-91e590fbb5c0
floor_id 5cd25751-5775-4002-a0b7-1c6ddffdd9f5
Action: Move Step
Success: True


In [5]:
from abstractions.goap.actions import Action, Prerequisites, Consequences
from abstractions.goap.entity import Attribute, Statement
from abstractions.goap.spatial import GameEntity, Node, Position, GridMap, ActionsPayload, ActionInstance, ActionsResults, Path, BlocksMovement, BlocksLight
from typing import List, Dict, Any, Optional
import random

from abstractions.goap.procedural import generate_dungeon

In [6]:
def generate_move_actions(source_id: str, path: Path) -> List[Tuple[ActionInstance, Position, Position]]:
    move_actions = []
    for i in range(len(path.nodes) - 1):
        source_node = path.nodes[i]
        target_node = path.nodes[i + 1]
       
        # Find a floor entity in the target node to use as the target for the MoveStep action
        floor_entities = [entity for entity in target_node.entities if entity.name.startswith("Floor")]
        if floor_entities:
            target_id = floor_entities[0].id
            move_action = ActionInstance(source_id=source_id, target_id=target_id, action=MoveStep())
            move_actions.append((move_action, source_node.position, target_node.position))
        else:
            raise ValueError(f"No floor entity found in target node {target_node}")
   
    return move_actions

# Create a larger grid map
grid_map = GridMap(width=50, height=50)

# Fill the grid map with walls
for x in range(grid_map.width):
    for y in range(grid_map.height):
        wall = GameEntity(name=f"Wall_{x}_{y}", blocks_movement=BlocksMovement(value=True), blocks_light=BlocksLight(value=True))
        grid_map.get_node((x, y)).add_entity(wall)

# Generate the dungeon
num_rooms = 10
min_room_size = 5
max_room_size = 10
generate_dungeon(grid_map, num_rooms, min_room_size, max_room_size)

# Find valid floor tiles for player and goal
floor_tiles = [(x, y) for x in range(grid_map.width) for y in range(grid_map.height) if any(isinstance(entity, GameEntity) and entity.name.startswith("Floor") for entity in grid_map.get_node((x, y)).entities)]
player_pos, goal_pos = random.sample(floor_tiles, 2)

# Place the player and goal entities
player = Character(name="Player", blocks_movement=BlocksMovement(value=False), blocks_light=BlocksLight(value=False))
goal = GameEntity(name="Goal")
grid_map.get_node(player_pos).add_entity(player)
grid_map.get_node(goal_pos).add_entity(goal)

# Precompute payloads
start_node = grid_map.get_node(player_pos)
goal_node = grid_map.get_node(goal_pos)
path = grid_map.a_star(start_node, goal_node)
print(f"Start position: {start_node.position.value}, Goal position: {goal_node.position.value}")
# Generate the move actions based on the path
move_actions = generate_move_actions(player.id, path)



# Create an ActionsPayload using the move actions
actions_payload = ActionsPayload(actions=[action for action, _, _ in move_actions])

# Apply the actions payload to the grid map
actions_results = grid_map.apply_actions_payload(actions_payload)

# Print the results
for result in actions_results.results:
    print(f"\nAction: {result.action_instance.action.name}")
    print(f"Success: {result.success}")
    if not result.success:
        print(f"Error: {result.error}")
print("final player position", GameEntity.get_instance(player.id).node.position.value)



Start position: (16, 31), Goal position: (27, 42)

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success: True

Action: Move Step
Success